In [1]:
from datetime import datetime
import json
import os

import boto3 
from botocore.exceptions import ClientError
import pandas as pd
import requests

In [2]:
sqs_queue = 'jacobs-first-sqs'
sqs = boto3.resource('sqs')

# Get the queue
queue = sqs.get_queue_by_name(QueueName=sqs_queue)

In [5]:
# Sending a normal message w/ no attributes
response = queue.send_message(MessageBody='world')

# The response is NOT a resource, but gives you a message ID and MD5
print(response.get('MessageId')) # will change everytime
print(response.get('MD5OfMessageBody')) # will not change if given the same input body

b96685d8-24c2-4359-9603-198dca0e07f4
7d793037a0760186574b0282f2f435e7


In [64]:
# message with attributes
queue.send_message(MessageBody='boto3', MessageAttributes={
    'Author': {
        'StringValue': 'jacob',
        'DataType': 'String'
    },
    'Timestamp': {
        'StringValue': str(datetime.now()),
        'DataType': 'String'
    },
    'Payload': {
        'StringValue': 'JSON',
        'DataType': 'String'
    }
})

{'MD5OfMessageBody': '6686853da3491a56c98917cc5c4ddea2',
 'MD5OfMessageAttributes': '07d8782ce8e5d67de569b33dc2bb99c0',
 'MessageId': 'a27d6ba1-0b37-44b9-b5c4-f210646592b1',
 'ResponseMetadata': {'RequestId': '23225a46-691c-5eb0-be62-0115c5aca21d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '23225a46-691c-5eb0-be62-0115c5aca21d',
   'date': 'Thu, 21 Jul 2022 19:35:12 GMT',
   'content-type': 'text/xml',
   'content-length': '459'},
  'RetryAttempts': 0}}

In [58]:
# Send multiple messages in 1 call
response = queue.send_messages(Entries=[
    {
        'Id': '1',
        'MessageBody': 'type: world'
    },
    {
        'Id': '2',
        'MessageBody': 'boto3',
        'MessageAttributes': {
            'Owner': {
                'StringValue': 'jaaacob',
                'DataType': 'String'
            },
            'Timestamp': {
                'StringValue': str(datetime.now()),
                'DataType': 'String'
            },
        }
    }
])

# # Print out any failures
# print(response.get('Failed'))

In [63]:
for message in queue.receive_messages():
    print(message.message_attributes)

In [76]:
# Processing Messages
for message in queue.receive_messages(MessageAttributeNames=['Timestamp']):
    # Get the custom author message attribute if it was set
    author_text = ''
    if message.message_attributes is not None:
        author_name = message.message_attributes.get('Owner').get('StringValue')
        timestamp = message.message_attributes.get('Timestamp').get('StringValue')
        print(timestamp)
        if author_name:
            author_text = ' ({0})'.format(author_name)

    # Print out the body and author (if set)
    print('Hello, {0}!{1}'.format(message.body, author_text))
    message.delete()

In [ ]:
# A dead letter queue is a queue that one or more OTHER queues can use for messages that do not get successfully processed.
# instead of losing the message when processing fails, the message gets sent to the dead letter queue.
# if you're using a SOURCE FIFO queue, and you want a dead letter queue for it, then that dead letter queue must also be a FIFO queue.
# queue-ception.

In [34]:
dict1 = {
        'Id': '2',
        'MessageBody': 'boto3',
        'MessageAttributes': {
            'Author': {
                'StringValue': 'jaaacob',
                'DataType': 'String'
            }
        }
    }

print(dict1.get('Id'))
print(dict1.get('MessageAttributes').get('Author').get('StringValue'))
print(dict1['MessageAttributes']['Author']['StringValue']) # I prefer this 1

if 'MessageAttributes' in dict1:
    print('Message has Message Attributes')
else:
    print('oops')

2
jaaacob
jaaacob
Message has Message Attributes
